# Learning a quadratic pseudo-metric from distance measurements

Recall that pseudo-metric is a generalization of a metric space in which the distance between two distinct points can be zero.
We are given a set of $N$ pairs of points in $\mathbf{R}^n$, $x_1, \ldots, x_N$, and $y_1, \ldots, y_N$, together with a set of distances $d_1, \ldots, d_N > 0$.
  The goal is to find (or estimate or learn) a quadratic pseudo-metric $d$
  $$d(x,y) =  \left( (x-y)^T P(x-y) \right)^{1/2},$$
  $P\in \mathbf{S}^n_{+}$, which approximates the given distances, i.e., $d(x_i, y_i) \approx d_i$. (The pseudo-metric $d$ is a metric only when $P \succ 0$; when $P\succeq 0$ is singular, it is a pseudo-metric.)
  
  To do this, we will choose $P\in \mathbf{S}^n_+$ that minimizes the mean squared error objective
  
  $$f(S)=\frac{1}{N}\sum_{i=1}^N (d_i - d(x_i,y_i))^2.$$
  
  ### Theoretical part.
  1. Show that the objective function $f$ is convex (Hint: expand the square and see what happens.)
  2. Show that the convex program $\text{minimize }f(S)$, $S\succeq 0$ can be expressed by an equivalent conic program with linear objective and a number of conic constraints using the $R^n_+$ (nonnegative orthant cone), $Q^n$ (second order cone), $Q_r^n$ (rotated second order cone), $S^n_+$ (positive semidefinite cone).
  
  ### Programming Part
  1. Solve the program $\text{minimize }f(S)$, $S\succeq 0$, preferably using a modelling package like ``cvxpy``. Note that "under the hood" your modelling package translates the program to the conic form in point 2. above.
  2. Use the obtained $P$ to measure the mean square error for the test data ``X_test``, ``Y_test``, ``d_test``.
  
---- 
*This exercise originates from "Additional Exercises" collection for Convex Optimization textbook of S. Boyd and L. Vandenberghe. Used under permission*

In [1]:
import cvxpy as cp
import numpy as np
from scipy import linalg as la

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# In this box we generate the input data

np.random.seed(5680)

n = 5 # Dimension
N = 100 # Number of samples

P = np.random.randn(n,n)
P = P.dot(P.T) + np.identity(n)
sqrtP = la.sqrtm(P)

x = np.random.randn(N,n)
y = np.random.randn(N,n)

d = np.linalg.norm(sqrtP.dot((x-y).T),axis=0)    # distances according to metric P
d = np.maximum(d+np.random.randn(N),0)           # add random noise

N_test = 10 # Samples for test set
X_test = np.random.randn(N_test,n)
Y_test = np.random.randn(N_test,n)
d_test = np.linalg.norm(sqrtP.dot((X_test-Y_test).T),axis=0)  # distances according to metric P
d_test = np.maximum(d_test+np.random.randn(N_test),0)         # add random noise


## Theorethical part

1.  From the lecture, we know that the domain of our function is convex. To prove that $f(S)$ is convex, it is enough to show that for each $i\in\{1,\ldots,N\}$, the component $(d_i - d(x_i,y_i))^2$ is convex (we can clearly ignore the constant $\frac{1}{N}$ as $N > 0$). Furthermore, $(d_i - d(x_i,y_i))^2 = d_i^2 + (x_i-y_i)^T P (x_i-y_i) - 2d_i \sqrt{ (x_i-y_i)^T P (x_i-y_i)}$.

    Thus, we need to prove the function $$g_i(S) = d_i^2 + (x_i-y_i)^T S (x_i-y_i) - 2d_i \sqrt{ (x_i-y_i)^T S (x_i-y_i)}$$ is convex.

    Component $d_i^2$, as a constant, is convex. $(x_i-y_i)^T S (x_i-y_i)$ is a linear function with respect to the variable S, so it is convex. Moreover, from the lecture, $- 2d_i \sqrt{ (x_i-y_i)^T S (x_i-y_i)}$ is convex as a composition of the convex non-increasing function $\left(h(x) = -x\right)$ with the concave function $\left(g(x) = \sqrt{x}\right)$. Thus, $g_i$ is convex.

    Therefore, the function $f$ is convex as a sum of convex functions.

2.  We can add new two variable $a,b \in\mathbb{R}^N$ and rewrite our program to minimize $\sum_{i=1}^{N} a_i - b_i$, where $(x_i-y_i)S   (x_i-y_i) \leq a_i$ and $-2d_i\sqrt{(x_i-y_i)S(x_i-y_i)} \leq -b_i \implies $   $ 4d_i^2 (x_i-y_i)S(x_i-y_i) \geq  b_i^2$ for $\newline i \in \{1,\ldots, N\}$. Therefore, our conic program will be:

    $$
    \text{minimize} \quad \sum_{i=1}^{N} a_i - b_i
    $$

    subject to

    $$
    (b_i, 4d_i^2(x_i-y_i)^T S (x_i-y_i),1) \succeq_{Q_r} 0, \hspace{3mm}\text{ where }  Q_r = \{(x,t,s)\in\mathbb{R^{3}} :\hspace{1mm} \| x \|^2_2\leq ts\} \text{ for } i\in \{1,\ldots,N\} \\
    a_i - (x_i-y_i)^T S (x_i-y_i) \succeq_{R_+} 0, \hspace{3mm} \text{ for } i\in \{1,\ldots,N\} \\
    S \succeq_{S^n_{+} 0}
    $$

## Programming part



In [3]:
S = cp.Variable((n,n), name='S', PSD=True)

z = x - y

obj = cp.Minimize(cp.sum([z[i].T @ S @ z[i] - 2 * d[i] * cp.sqrt(z[i].T @ S @ z[i]) for i in range(N)]))

prob = cp.Problem(obj)

prob.solve(verbose=True,solver=cp.SCS)
print(f"Min f(S) {(prob.value + np.sum([d[i]**2 for i in range(N)])) / N}")

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) May 22 11:33:10 PM: Your problem has 25 variables, 0 constraints, and 0 parameters.


(CVXPY) May 22 11:33:10 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 22 11:33:10 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 22 11:33:10 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 22 11:33:10 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 22 11:33:10 PM: Compiling problem (target solver=SCS).
(CVXPY) May 22 11:33:10 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCS
(CVXPY) May 22 11:33:10 PM: Applying reduction Dcp2Cone
(CVXPY) May 22 11:33:10 PM: Applying reduction CvxAttr2Constr
(CVXPY) May 22 11:33:10 PM: Applying r

In [4]:
# Measure the mean square error for the test data X_test, Y_test, d_test.

z_test = X_test - Y_test

mse = (np.sum([(d_test[i] - np.sqrt(z_test[i].T @ S.value @ z_test[i]))**2 for i in range(N_test)])) / N_test
print(f"Mean squared distance error : {mse}")

Mean squared distance error : 0.9159307075239894
